In [ ]:
%autoreload 2

In [ ]:
from transformers import *
from PIL import Image
from dit import *
from warmup import *

In [ ]:
ocr_model = "microsoft/trocr-base-handwritten"
layoutlm_model = "microsoft/layoutlmv2-base-uncased"
from detectron2.data import *
from detectron2.utils.visualizer import *

In [ ]:
processor = TrOCRProcessor.from_pretrained(ocr_model)
model = VisionEncoderDecoderModel.from_pretrained(ocr_model)

In [ ]:
im = Image.open("./samples/p2.png")
fname = im.filename
im = im.convert("RGB")
im

In [ ]:
pixel_values = processor(im, return_tensors="pt").pixel_values
gen = model.generate(pixel_values)
gen

In [ ]:
processor.batch_decode(gen, skip_special_tokens=True)[0]

In [ ]:
import easyocr

reader = easyocr.Reader(["en"])
# text = [i[1] for i in reader.readtext("./samples/segment.png")]
# text

In [ ]:
doc_segmentation(im, visualize=True)

In [ ]:
cfg = get_cfg()
predictor = get_predictor(cfg)

md = MetadataCatalog.get(cfg.DATASETS.TEST[0])
output = predictor(np.array(im))["instances"]

In [ ]:
text = []

In [ ]:
classes = output.get("pred_classes")
boxes = output.get("pred_boxes")

In [ ]:
for cls, box in iter(zip(classes, boxes)):
    print(cls.numpy(), box.numpy())

In [ ]:
for cls, box in zip(classes, boxes):
    if cls.numpy() == 0:  # 0 is the text block only
        cropped = im.crop(box.numpy())
        # cropped.show()
        infer_text = "".join([t[1] for t in reader.readtext(np.array(cropped))])
        if not infer_text.startswith("Figure"):
            print(infer_text)
            text.append("".join(infer_text))

In [ ]:
it = next(iter(boxes))
cropped = im.crop(it.numpy())
cropped

In [ ]:
text

In [ ]:
import pdf2image

In [ ]:
images = pdf2image.convert_from_path("./samples/2204.08387.pdf")  # layoutlmv3 paper
images

In [ ]:
import subprocess

subprocess.check_output(["pdfinfo", "-v"], stderr=subprocess.PIPE)

In [ ]:
image = images[0]
classes, scores, boxes = doc_segmentation(image.convert("RGB"), visualize=True)

In [ ]:
output = predictor(np.asarray(image.convert("RGB")))["instances"]

In [ ]:
output.get("scores").tolist()

In [ ]:
crops = [im.crop(b.numpy()) for b in boxes]
crops

In [ ]:
import cloudpickle

with open("reader.pkl", "wb") as f:
    cloudpickle.dump(reader, f)

In [ ]:
with open("reader.pkl", "rb") as f:
    loaded = cloudpickle.load(f)

In [ ]:
loaded.readtext(np.asarray(image))